In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Jun 10 14:14:04 2022

@author: davidnaranjo
"""
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
# Importing the m importlibain code.
module_path = os.path.abspath('/Users/localadmin/Dropbox/GitHub/ocloc/src/ocloc')

if module_path not in sys.path:
    sys.path.append(module_path)
import ocloc
from ocloc import ProcessingParameters, ClockDrift, trim_correlation_trace, suppress_stdout
import pickle
from statistics import mean
# Parameters for locating the files where the correlation files and station 
# information is contained.
path2data_dir = "/Users/localadmin/Dropbox/GitHub/data"
# path2data_dir = "/Users/localadmin/Dropbox/GitHub/ocloc/tutorials/correlations_O20"
# station_file = "/Users/localadmin/Dropbox/GitHub/ocloc/tutorials/station_info"
station_file = "/Users/localadmin/Dropbox/GitHub/ocloc/tutorials/metadata/station_file.txt"

reference_time = '2014-08-21T00:00:00.000000Z'

In [3]:
def solve_inverse(dist_trh, snr_trh):
    """
    Fuinction to calculate the a's and b's of a station using
    different dist_trh and snr_trh.
    
    Returns ClockDrift object which save everything needed.
    """
    params = ProcessingParameters(
        freqmin=0.2,   # Low freq. for the bandpass filter
        freqmax=0.4,   # High freq. for the bandpass filter 
        ref_vel=2500,  # m/s
        dist_trh=dist_trh,  # Minimum station separation in terms of wavelength
        snr_trh=snr_trh    # Signal-to-noise ratio threshold
        )

    cd = ClockDrift(station_file, path2data_dir, 
                    reference_time=reference_time,
                    list_of_processing_parameters=[params])
    with suppress_stdout():
        cd.calculate_aprioridt_4_allcorrelations()
        cd.calculate_dt_ins()
        cd.calculate_tapp_4_allcorrelations()

        for i in range(3):
            cd.filter_stations()
            cd.build_matrices()
            cd.solve_eq(method="weighted_lstsq")
            cd.calculate_dt_ins()
            cd.calculate_tapp_4_allcorrelations()

    # Make a few plots
    plt.show()
    print("snr : " + str(snr_trh) + "\n distance threshold: " + str(dist_trh))
    #cd.plot_matrix_diff_observed_predicted()
    #cd.plot_fluctuation_of_a_and_b()
    #cd.plot_observed_vs_predicted()
    return cd




In [4]:
results = {}
for snr_trh in [20, 30, 40, 50, 60, 10]:
    for dist_trh in [2.5, 3.0, 3.5, 4.0, 1.5, 2.0]:
        iteration = "snr_trh_" + str(snr_trh) + "__dist_trh_" + str(dist_trh)
        results["weighted a " + iteration] = []
        results["weighted b " + iteration] = []
        results["lstsq a " + iteration] = []
        results["lstsq b " + iteration] = []

        cd = solve_inverse(dist_trh, snr_trh)
        
        f = open("ClockDrift_"+iteration+".obj", "wb")
        pickle.dump(results, f)
        f.close()

No correlation file found for station:#STK
snr : 20
 distance threshold: 2.5
No correlation file found for station:#STK
snr : 20
 distance threshold: 3.0
No correlation file found for station:#STK
snr : 20
 distance threshold: 3.5
No correlation file found for station:#STK
snr : 20
 distance threshold: 4.0
No correlation file found for station:#STK
snr : 20
 distance threshold: 1.5
No correlation file found for station:#STK
snr : 20
 distance threshold: 2.0
No correlation file found for station:#STK
snr : 30
 distance threshold: 2.5
No correlation file found for station:#STK
snr : 30
 distance threshold: 3.0
No correlation file found for station:#STK
snr : 30
 distance threshold: 3.5
No correlation file found for station:#STK
snr : 30
 distance threshold: 4.0
No correlation file found for station:#STK
snr : 30
 distance threshold: 1.5
No correlation file found for station:#STK
snr : 30
 distance threshold: 2.0
No correlation file found for station:#STK
snr : 40
 distance threshold: 2.5

KeyboardInterrupt: 